In [249]:
import pandas as pd
import glob
import os
import tqdm
import gc
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import seaborn as sns
import numpy as np
from scipy.spatial import Voronoi, voronoi_plot_2d
import math
from sklearn import datasets, linear_model, model_selection
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix
from tqdm import tqdm

> ДЗ 1 Классификация Домашнее задание из двух частей. (после НГ)
> База про спортсменов
> 
> Креативная часть: найти парамеры и комбинации их, которые будут увеличивать accuracy
> 
> Техническая часть: сделать под это всё удобные обёртки. Например, это функция, которая просто принимает список фичей, а на выходе показывает результат фита

In [250]:
df = pd.read_csv("/kaggle/input/athletes-sochi/athletes_sochi.txt").dropna()
df

,age,birthdate,gender,height,name,weight,gold_medals,silver_medals,bronze_medals,total_medals,sport,country
0,17,1996-04-12,Male,1.72,Aaron Blunck,68.0,0,0,0,0,Freestyle Skiing,United States
1,27,1986-05-14,Male,1.85,Aaron March,85.0,0,0,0,0,Snowboard,Italy
2,21,1992-06-30,Male,1.78,Abzal Azhgaliyev,68.0,0,0,0,0,Short Track,Kazakhstan
4,21,1992-07-30,Male,1.86,Adam Barwood,82.0,0,0,0,0,Alpine Skiing,New Zealand
5,21,1992-12-18,Male,1.75,Adam Cieslar,57.0,0,0,0,0,Nordic Combined,Poland
...,...,...,...,...,...,...,...,...,...,...,...,...
2851,28,1985-04-30,Male,1.93,Ziga Pavlin,98.0,0,0,0,0,Ice Hockey,Slovenia
2853,31,1982-12-05,Female,1.70,Zina Kocher,60.0,0,0,0,0,Biathlon,Canada
2854,28,1985-06-14,Female,1.68,Zoe Gillings,65.0,0,0,0,0,Snowboard,Great Britain
2856,22,1991-03-01,Male,1.76,Zongyang Jia,68.0,0,0,1,1,Freestyle Skiing,China


In [251]:
def classify(features, df):
    df = df[features]

    gender_binar = {"gender": {"Male" : 1, "Female" : 0}}
    df = df.replace(gender_binar)
    
    features.remove('gender')
        
    allinput = df[features]
    alloutput = df[['gender']]

    print(allinput)
    print(alloutput)
    
    trainAndAccessModel(allinput, alloutput)
    
    
    
def trainAndAccessModel(allinput, alloutput):
    chunk_80 = int(len(df) * 0.8)

    X_train = allinput[ :chunk_80]
    Y_train = alloutput[ :chunk_80]

    X_test = allinput[chunk_80: ]
    Y_test = alloutput[chunk_80: ]

    #Создаем модель
    model = linear_model.LogisticRegression()

    #Обучаем модель
    model.fit(X_train, Y_train)

    print("Коэфициенты модели: ", model.coef_)
    
    Y_test_predicted = model.predict(X_test)
    
    [tn, fp], [fn, tp] = confusion_matrix(Y_test, Y_test_predicted)
    
    print("Accuracy: ", (tn + tp) / (tp + fp + tn + fn))
    



Комбинация параметров возраст+рост+вес:

In [252]:
classify(['age', 'height', 'weight', 'gender'], df)

      age  height  weight
0      17    1.72    68.0
1      27    1.85    85.0
2      21    1.78    68.0
4      21    1.86    82.0
5      21    1.75    57.0
...   ...     ...     ...
2851   28    1.93    98.0
2853   31    1.70    60.0
2854   28    1.68    65.0
2856   22    1.76    68.0
2857   19    1.58    54.0

[2479 rows x 3 columns]
      gender
0          1
1          1
2          1
4          1
5          1
...      ...
2851       1
2853       0
2854       0
2856       1
2857       0

[2479 rows x 1 columns]
Коэфициенты модели:  [[-0.01251138  7.13590742  0.16183555]]
Accuracy:  0.8770161290322581


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Комбинация параметров возраст+рост:

In [253]:
classify(['age', 'height', 'gender'], df)

      age  height
0      17    1.72
1      27    1.85
2      21    1.78
4      21    1.86
5      21    1.75
...   ...     ...
2851   28    1.93
2853   31    1.70
2854   28    1.68
2856   22    1.76
2857   19    1.58

[2479 rows x 2 columns]
      gender
0          1
1          1
2          1
4          1
5          1
...      ...
2851       1
2853       0
2854       0
2856       1
2857       0

[2479 rows x 1 columns]
Коэфициенты модели:  [[ 0.02669006 15.77414289]]
Accuracy:  0.8467741935483871


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Вывод: комбинация ['age', 'height', 'weight', 'gender'] дает лучшую accurency